In [1]:
import numpy as np
import pandas as pd

In [2]:
alldata = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick202004final.xlsx")
data1 = alldata.groupby(['LocationID']).mean()
data1 = data1[['Freq','pop_density','commercial_density','road_density','X','Y']]

from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data1[['pop_density','commercial_density','road_density']])
data1[['pop_density','commercial_density','road_density']] = scaler.transform(data1[['pop_density','commercial_density','road_density']])

In [16]:
X = np.array(data1[["X"]])
Y = np.array(data1[["Y"]])
coords = np.hstack([X,Y])


y = np.array(data1[["Freq"]])

pop = np.array(data1[["pop_density"]])
com = np.array(data1[["commercial_density"]])
roa = np.array(data1[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.sel import search_GWRparameter

sel = search_GWRparameter(coords, y, x, kernel = 'bisquare')

bw = sel.search(verbose = True)

bw: 126.0 , score: 1859.54
bw: 96.0 , score: 1821.61
bw: 78.0 , score: 1805.46
bw: 66.0 , score: 1801.38
bw: 59.0 , score: 1800.64
bw: 59.0 , score: 1800.64
bw: 62.0 , score: 1799.5
bw: 62.0 , score: 1799.5
bw: 62.0 , score: 1799.5
bw: 62.0 , score: 1799.5


In [3]:
X = np.array(data1[["X"]])
Y = np.array(data1[["Y"]])
coords = np.hstack([X,Y])

y = np.array(data1[["Freq"]])

pop = np.array(data1[["pop_density"]])
com = np.array(data1[["commercial_density"]])
roa = np.array(data1[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.model import GWR

results1 = GWR(coords, y, x,49, kernel='bisquare').fit()

results1.R2

0.851768616673437

In [18]:
coords = results1.coords
coords = pd.DataFrame(coords)

betas = results1.betas
betas = pd.DataFrame(betas)

tvalues = results1.tvalues
tvalues = pd.DataFrame(tvalues)

final_result = pd.concat([betas,tvalues,coords], axis = 1)
final_result.columns=['beta0','POP_beta','COM_beta','ROA_beta','t_beta0','t_POP_beta','t_COM_beta','t_ROA_beta','X','Y']

data2 = pd.merge(data1, final_result, left_on=['X','Y'], right_on=['X','Y'] , how='left')
data2.drop(['Freq','pop_density','commercial_density','road_density'],axis=1,inplace = True)
data2.to_csv(r'E:\\Capstone Project\\Data\\paper\\minor\\pick202004beta.csv',index=True,sep=',')

In [4]:
betas = results1.betas
betas = pd.DataFrame(betas)
betas.describe()

,0,1,2,3
count,252.000000,252.000000,252.000000,252.000000
mean,6.488720,0.825961,0.966944,0.306753
std,9.433952,2.799537,2.498921,3.110099
min,-6.965150,-4.762119,-6.094518,-16.909585
25%,1.685253,-0.292521,-0.119516,-0.489198
50%,2.225269,0.095883,0.217532,-0.040165
75%,3.552732,0.456166,1.050017,0.591702
max,39.675993,13.741290,15.527139,13.277937
